In [1]:
import pandas as pd
from datetime import datetime
# from load_and_transform import check_winner, calc_score #player_list, team_list, playerDF, betsDF, scoringDF, playoffDF

def check_winner(row): # winner column in game Data
    if row["Score Home"] > row["Score Guest"]:
        return 1
    else:
        return 2
def calc_score(row): # check if player guessed correctly
    if row[player] == row["Winner"]:
        return 1
    else:
        return 0

thisDay = datetime.today().strftime("%Y-%m-%d")
thisDay = datetime(2024, 9, 6)

betsDF = pd.read_csv("data/bets.csv", delimiter=";")
betsDF["Date"] = pd.to_datetime(betsDF["Date"], format="%d.%m.%Y %H:%M")
filtered_bets = betsDF.loc[betsDF["Date"]<thisDay].copy()
thisWeek = filtered_bets["Round Number"].max()
lastWeek = thisWeek-1

# playoffDF = pd.read_csv("season_2024/data/playoffTipps.csv", delimiter=";")
# superbowlDF = pd.read_csv("season_2024/data/superbowl.csv", delimiter=";")
# player List of all participants and list of all teams
player_list = ["Alex", "Alina", "Evelyn", "Christopher", "Ludwig", "Manu", "Natalie", "Nikolai", "Sebastian", "Vero", "Viki", "Wolfgang"]
team_list = betsDF["Home Team"].unique()

# create additional Dataframes
playerDF = pd.DataFrame(data={"Player":player_list}) # sums up 
scoringDF = pd.DataFrame(columns=player_list) # collect the points for each game
scoringDF["Week Count"] = filtered_bets["Round Number"]
# playoffScoresDF = pd.DataFrame(columns=player_list) # playoff pre-bets
# playoffScoresDF["Teams"] = playoffDF["Teams"]
# playoffScoresDF["Division"] = playoffDF["Division"]

# Dataframes for Team evaluation
lucky_teamDF = pd.DataFrame(columns=player_list)
unlucky_teamDF = pd.DataFrame(columns=player_list)
bets_per_teamDF = pd.DataFrame(columns=player_list)

# Transform Dataframes
filtered_bets["Winner"] = filtered_bets.apply(check_winner, axis=1)



# st.dataframe(filtered_bets)

In [2]:
betsDF

,Round Number_2,Match Number,Round Number,Game Nr.,Date,Location,Home Team,Away Team,Score Home,Score Guest,...,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,Viki,Wolfgang
0,1,16,1,1,2023-09-08 00:20:00,GEHA Field at Arrowhead Stadium,Kansas City Chiefs,Detroit Lions,20,21,...,1,NaN,NaN,NaN,NaN,1.0,NaN,1,1.0,1
1,1,8,1,2,2023-09-10 17:00:00,Mercedes-Benz Stadium,Atlanta Falcons,Carolina Panthers,24,10,...,1,NaN,NaN,NaN,NaN,1.0,NaN,1,1.0,1
2,1,9,1,3,2023-09-10 17:00:00,M&T Bank Stadium,Baltimore Ravens,Houston Texans,25,9,...,1,NaN,NaN,NaN,NaN,1.0,NaN,1,2.0,1
3,1,10,1,4,2023-09-10 17:00:00,Cleveland Browns Stadium,Cleveland Browns,Cincinnati Bengals,24,3,...,2,NaN,NaN,NaN,NaN,2.0,NaN,2,1.0,2
4,1,11,1,5,2023-09-10 17:00:00,Lucas Oil Stadium,Indianapolis Colts,Jacksonville Jaguars,21,31,...,2,NaN,NaN,NaN,NaN,1.0,NaN,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,20,281,20,3,2024-01-21 21:00:00,Ford Field,Detroit Lions,Tampa Bay Buccaneers,31,23,...,1,1.0,1.0,1.0,2.0,1.0,1.0,1,1.0,1
281,20,282,20,4,2024-01-22 00:30:00,Highmark Stadium,Buffalo Bills,Kansas City Chiefs,24,27,...,1,2.0,1.0,2.0,2.0,2.0,2.0,1,1.0,2
282,21,283,21,1,2024-01-28 21:00:00,M&T Bank Stadium,Baltimore Ravens,Kansas City Chiefs,10,17,...,1,2.0,1.0,1.0,NaN,2.0,1.0,1,1.0,1
283,21,284,21,2,2024-01-29 00:30:00,Levi's® Stadium,San Francisco 49ers,Detroit Lions,34,31,...,1,1.0,2.0,2.0,NaN,1.0,2.0,1,2.0,1


In [3]:
schedule = pd.read_csv("data/schedule.csv", delimiter=";")
schedule["Date"] = pd.to_datetime(schedule["Date"], format="%Y.%m.%d %H:%M")
schedule

,Counter,Game Nr.,Week,Home Team,Away Team,Location,Date
0,1,1,1,Kansas City Chiefs,Baltimore Ravens,"GEHA Field at Arrowhead Stadium, Kansas City, MO",2024-09-05 20:20:00
1,2,2,1,Philadelphia Eagles,Green Bay Packers,"Corinthians Arena, Sao Paulo",2024-09-06 20:15:00
2,3,3,1,Atlanta Falcons,Pittsburgh Steelers,"Mercedes-Benz Stadium, Atlanta, GA",2024-09-08 13:00:00
3,4,4,1,Buffalo Bills,Arizona Cardinals,"Highmark Stadium, Orchard Park, NY",2024-09-08 13:00:00
4,5,5,1,Chicago Bears,Tennessee Titans,"Soldier Field, Chicago, IL",2024-09-08 13:00:00
...,...,...,...,...,...,...,...
267,268,12,18,Philadelphia Eagles,New York Giants,"Lincoln Financial Field, Philadelphia, PA",2025-01-16 00:00:00
268,269,13,18,Arizona Cardinals,San Francisco 49ers,"State Farm Stadium, Glendale, AZ",2025-01-17 00:00:00
269,270,14,18,Pittsburgh Steelers,Cincinnati Bengals,"Acrisure Stadium, Pittsburgh, PA",2025-01-18 00:00:00
270,271,15,18,Tampa Bay Buccaneers,New Orleans Saints,"Raymond James Stadium, Tampa, FL",2025-01-19 00:00:00


In [4]:
thisWeek = 1

thisWeek_DF = schedule.loc[schedule["Week"]==thisWeek, ["Game Nr.", "Date", "Location", "Home Team", "Away Team"]]
lastWeek_DF = betsDF.loc[betsDF["Round Number"]==lastWeek, ["Game Nr.", "Home Team", "Score Home", "Score Guest", "Away Team"]]

In [5]:
thisWeek_DF

,Game Nr.,Date,Location,Home Team,Away Team
0,1,2024-09-05 20:20:00,"GEHA Field at Arrowhead Stadium, Kansas City, MO",Kansas City Chiefs,Baltimore Ravens
1,2,2024-09-06 20:15:00,"Corinthians Arena, Sao Paulo",Philadelphia Eagles,Green Bay Packers
2,3,2024-09-08 13:00:00,"Mercedes-Benz Stadium, Atlanta, GA",Atlanta Falcons,Pittsburgh Steelers
3,4,2024-09-08 13:00:00,"Highmark Stadium, Orchard Park, NY",Buffalo Bills,Arizona Cardinals
4,5,2024-09-08 13:00:00,"Soldier Field, Chicago, IL",Chicago Bears,Tennessee Titans
5,6,2024-09-08 13:00:00,"Paycor Stadium, Cincinnati, OH",Cincinnati Bengals,New England Patriots
6,7,2024-09-08 13:00:00,"Lucas Oil Stadium, Indianapolis, IN",Indianapolis Colts,Houston Texans
7,8,2024-09-08 13:00:00,"Hard Rock Stadium, Miami Gardens, FL",Miami Dolphins,Jacksonville Jaguars
8,9,2024-09-08 13:00:00,"Caesars Superdome, New Orleans, LA",New Orleans Saints,Carolina Panthers
9,10,2024-09-08 13:00:00,"MetLife Stadium, East Rutherford, NJ",New York Giants,Minnesota Vikings


In [6]:
list(schedule.loc[schedule["Date"]<"2024-10-13", "Week"])[-1]

6

In [7]:
submitted_data = ["Kansas City Chiefs","Philadelphia Eagles","Atlanta Falcons","Buffalo Bills","Chicago Bears","Cincinnati Bengals","Indianapolis Colts","Miami Dolphins",
                  "New Orleans Saints","New York Giants","Los Angeles Chargers","Seattle Seahawks","Cleveland Browns","Tampa Bay Buccaneers","Los Angeles Rams",
                  "San Francisco 49ers","Alex"
]

In [8]:
thisWeek_DF[f"{submitted_data[-1]}"] = submitted_data[:-1]
thisWeek_DF

,Game Nr.,Date,Location,Home Team,Away Team,Alex
0,1,2024-09-05 20:20:00,"GEHA Field at Arrowhead Stadium, Kansas City, MO",Kansas City Chiefs,Baltimore Ravens,Kansas City Chiefs
1,2,2024-09-06 20:15:00,"Corinthians Arena, Sao Paulo",Philadelphia Eagles,Green Bay Packers,Philadelphia Eagles
2,3,2024-09-08 13:00:00,"Mercedes-Benz Stadium, Atlanta, GA",Atlanta Falcons,Pittsburgh Steelers,Atlanta Falcons
3,4,2024-09-08 13:00:00,"Highmark Stadium, Orchard Park, NY",Buffalo Bills,Arizona Cardinals,Buffalo Bills
4,5,2024-09-08 13:00:00,"Soldier Field, Chicago, IL",Chicago Bears,Tennessee Titans,Chicago Bears
5,6,2024-09-08 13:00:00,"Paycor Stadium, Cincinnati, OH",Cincinnati Bengals,New England Patriots,Cincinnati Bengals
6,7,2024-09-08 13:00:00,"Lucas Oil Stadium, Indianapolis, IN",Indianapolis Colts,Houston Texans,Indianapolis Colts
7,8,2024-09-08 13:00:00,"Hard Rock Stadium, Miami Gardens, FL",Miami Dolphins,Jacksonville Jaguars,Miami Dolphins
8,9,2024-09-08 13:00:00,"Caesars Superdome, New Orleans, LA",New Orleans Saints,Carolina Panthers,New Orleans Saints
9,10,2024-09-08 13:00:00,"MetLife Stadium, East Rutherford, NJ",New York Giants,Minnesota Vikings,New York Giants


In [9]:
submitted_data = ["Kansas City Chiefs","Philadelphia Eagles","Atlanta Falcons","Buffalo Bills","Chicago Bears","Cincinnati Bengals","Indianapolis Colts","Miami Dolphins",
                  "New Orleans Saints","New York Giants","Los Angeles Chargers","Seattle Seahawks","Cleveland Browns","Tampa Bay Buccaneers","Los Angeles Rams",
                  "San Francisco 49ers","Alina"
]
thisWeek_DF[f"{submitted_data[-1]}"] = submitted_data[:-1]
thisWeek_DF

,Game Nr.,Date,Location,Home Team,Away Team,Alex,Alina
0,1,2024-09-05 20:20:00,"GEHA Field at Arrowhead Stadium, Kansas City, MO",Kansas City Chiefs,Baltimore Ravens,Kansas City Chiefs,Kansas City Chiefs
1,2,2024-09-06 20:15:00,"Corinthians Arena, Sao Paulo",Philadelphia Eagles,Green Bay Packers,Philadelphia Eagles,Philadelphia Eagles
2,3,2024-09-08 13:00:00,"Mercedes-Benz Stadium, Atlanta, GA",Atlanta Falcons,Pittsburgh Steelers,Atlanta Falcons,Atlanta Falcons
3,4,2024-09-08 13:00:00,"Highmark Stadium, Orchard Park, NY",Buffalo Bills,Arizona Cardinals,Buffalo Bills,Buffalo Bills
4,5,2024-09-08 13:00:00,"Soldier Field, Chicago, IL",Chicago Bears,Tennessee Titans,Chicago Bears,Chicago Bears
5,6,2024-09-08 13:00:00,"Paycor Stadium, Cincinnati, OH",Cincinnati Bengals,New England Patriots,Cincinnati Bengals,Cincinnati Bengals
6,7,2024-09-08 13:00:00,"Lucas Oil Stadium, Indianapolis, IN",Indianapolis Colts,Houston Texans,Indianapolis Colts,Indianapolis Colts
7,8,2024-09-08 13:00:00,"Hard Rock Stadium, Miami Gardens, FL",Miami Dolphins,Jacksonville Jaguars,Miami Dolphins,Miami Dolphins
8,9,2024-09-08 13:00:00,"Caesars Superdome, New Orleans, LA",New Orleans Saints,Carolina Panthers,New Orleans Saints,New Orleans Saints
9,10,2024-09-08 13:00:00,"MetLife Stadium, East Rutherford, NJ",New York Giants,Minnesota Vikings,New York Giants,New York Giants


In [16]:
from datetime import datetime

datetime.now().strftime("%Y-%m-%d %H:%M:%S,%f")

'2024-08-14 11:16:42,083306'